In [1]:
%pip install bertopic
%pip install seaborn
%pip install spacy
%pip install nltk
%pip install pandas nltk spacy
!python -m spacy download es_core_news_sm
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.9 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 56.0 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 69.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 38.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 60.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 39.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 33.5 MB/s  0:00:00m0:00:01:00:01
  Created wheel for hdbscan: filename=hdbscan-0.8.40-cp312-cp312-linux_x86_64.whl size=4604516 sha256=b8fdcb985f222cffd45caad38955224afe8be9682fb79784e57fa

In [ ]:
import pandas as pd

# Read CSV
df_contratos = pd.read_csv("file_path")
# Display the first few rows
print(df_contratos.head())

In [ ]:
#Changing the datatypes


# Assuming data is your DataFrame

# 1. List all text/string columns
text_columns = [
    'objeto_del_contrato',
    'duraci_n_del_contrato',
    'urlproceso',
    'nombre_representante_legal',
    'origen_de_los_recursos',
    'liquidaci_n',
    'es_pyme',
    'proveedor_adjudicado',
    'modalidad_de_contratacion',
    'tipo_de_contrato',
    'codigo_de_categoria_principal',
    'descripcion_del_proceso',
    'estado_contrato',
    'referencia_del_contrato',
    'id_contrato',
    'proceso_de_compra',
    'entidad_centralizada',
    'rama',
    'sector',
    'orden',
    'localizaci_n',
    'ciudad',
    'departamento',
    'nit_entidad',
    'nombre_entidad'
]

# 2. List all numeric (float) columns
numeric_columns = [
    'presupuesto_general_de_la_nacion_pgn',
    'recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
    'valor_del_contrato',
    'valor_facturado'
]

# 3. List all date columns
date_columns = [
    'fecha_inicio_liquidacion',
    'fecha_fin_liquidacion',
    'fecha_de_fin_de_ejecucion',
    'fecha_de_inicio_de_ejecucion',
    'fecha_de_fin_del_contrato',
    'fecha_de_inicio_del_contrato',
    'fecha_de_firma'
]

# Convert text columns to string
for col in text_columns:
    if col in df_contratos.columns:
        df_contratos[col] = df_contratos[col].astype(str)

# Convert numeric columns to float (handling errors)
for col in numeric_columns:
    if col in df_contratos.columns:
        df_contratos[col] = pd.to_numeric(df_contratos[col], errors='coerce')

# Convert date columns to datetime (handling errors)
for col in date_columns:
    if col in df_contratos.columns:
        df_contratos[col] = pd.to_datetime(df_contratos[col], errors='coerce')


# Show the final data types
print("Final Data Types:")
print(df_contratos.dtypes)

In [ ]:
##optional, counting the ammount of contracts by month

# ---------- Process fecha_de_firma ----------
df_contratos['fecha_de_firma'] = pd.to_datetime(df_contratos['fecha_de_firma'], errors='coerce')

# Drop rows with invalid dates just in case
df_contratos = df_contratos.dropna(subset=['fecha_de_firma'])

# Group by month
df_contratos['mes'] = df_contratos['fecha_de_firma'].dt.to_period('M')
conteo_mensual = df_contratos.groupby('mes').size().reset_index(name='conteo')

# ---------- Output ----------
print(conteo_mensual.to_string(index=False))  # Ensures all rows are shown

In [ ]:
# Sample 1000 rows per month (or all if fewer exist)
sampled_df = (
    df_contratos
    .groupby('mes', group_keys=False)  # Group by month without adding extra index
    .apply(lambda x: x.sample(n=min(10000, len(x)), random_state=42))  # Sample up to 1000
    .reset_index(drop=True)  # Reset index for clean output

)

# Verify the counts per month
print(sampled_df['mes'].value_counts().sort_index())

# (Optional) Save to CSV
# sampled_df.to_csv('sampled_contracts.csv', index=False)

In [ ]:
## Cleaning, removing stop words
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy

# Load Spanish stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# 👉 Add domain-specific stopwords (based on your exploration)
contract_stopwords = {
    'prestacion', 'servicio', 'proceso', 'contratista', 'acuerdo', 'ejecutar',
    'condicion', 'empresa', 'nuevo', 'institucional', 'entidad', 'area',
    'eficaz', 'eficiente', 'municipio', 'nivel', 'centro', 'ejecución',
    'prestación', 'objeto', 'dentro', 'requerido', 'diferente', 'poner',
    'realizar',
    'apoyo', 'departamental', 'contrato', 'laboral', 'actividad', 'cuenta',
    'profesional', 'contratar', 'especializado', 'prestar', 'área',
    'subproceso', 'comprometer', 'garantizar', 'requerir', 'adelantar',
    'local', 'primero', 'conformidad', 'responsabilidad', 'efectivo',
    'disposición', 'forma', 'propuesta', 'bajo', 'oportuno', 'tiempo', 'autonomía'
}
stop_words.update(contract_stopwords)

# Load Spanish language model from spaCy
nlp = spacy.load("es_core_news_sm")

# Define full cleaning function with lemmatization and entity removal
def clean_lemma_no_entities(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if token.lemma_ not in stop_words
        and token.ent_type_ not in ("PER", "LOC", "ORG")  # Named entities
        and not token.is_punct and not token.is_space
    ]
    return ' '.join(tokens)

# Apply to your contract descriptions
df =sampled_df.copy()
df['objeto_clean_lemma'] = df['objeto_del_contrato'].apply(clean_lemma_no_entities)

In [ ]:
###NEW BERTOPIC MODEL

# 1. Import libraries
import umap
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# 2. Define custom UMAP model
umap_model = umap.UMAP(
    n_neighbors=15,
    min_dist=0.0001,
    n_components=2,  # Reduced dimensionality
    metric='cosine',
    random_state=42
)

# 3. Define HDBSCAN clusterer
clusterer = HDBSCAN(
    min_cluster_size=30,
    min_samples=15,
    metric='euclidean',  # Matches UMAP's output space
    prediction_data=True  # Essential for future predictions
)

# 4. Create BERTopic model
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=clusterer,  # Added custom clusterer
    language="spanish",
    verbose=True
)

# 5. Run the model
docs = df['objeto_clean_lemma'].tolist()
topics, probs = topic_model.fit_transform(docs)

In [ ]:
# 7. (Optional) Validate and explore topics
print(topic_model.get_topic_info().head(10))  # Top topics
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
# Save entire model to disk (including embeddings and parameters)
topic_model.save("my_bertopic_modelv07", serialization="safetensors", save_ctfidf=True)


In [ ]:
from bertopic import BERTopic

# Load model
topic_model = BERTopic.load("my_bertopic_modelv07",embedding_model="all-MiniLM-L6-v2")



In [ ]:
!pip install chardet
import chardet

with open("G:/Mi unidad/Colab Notebooks/Licitaciones/Cursor Venv v2/topicsv07-trial.csv", "rb") as f:
    result = chardet.detect(f.read(10000))  # Read a portion of the file for detection

print(result)

In [ ]:
new_docs = df_nueva['objeto_clean_lemma'].tolist()

In [ ]:
# Predict topics using the loaded model
new_topics, new_probs = topic_model.transform(new_docs)

In [ ]:
# Add predictions to your new_df
df_nueva['predicted_topic'] = new_topics
df_nueva['topic_probability'] = new_probs

In [ ]:
# (Optional) save predictions
df_nueva.to_csv("new_feedbacks_with_topics.csv", index=False)

# View a few examples
print(df_nueva[['objeto_clean_lemma', 'predicted_topic', 'topic_probability']].head())